# PySparkに慣れる

## インストール
基本は[Quickstart](https://spark.apache.org/docs/latest/api/python/getting_started/testing_pyspark.html)に従って、  
pip install pysparkすればインストールできる。  
ただし、jdkが入っていない場合はjdkもインストールしておかないとJAVA_HOMEが設定されていないというエラーが起きる。  
インストール後は特に明示的にJAVA‗HOMEを設定しなくても大丈夫だったが、場合によっては環境変数にパスを設定しておく必要があるかも。

## 動作テスト
文法はPolarsに似ている。（というかPolarsがPySparkに似ている？）  
ノートPCでの動作はPolarsよりも遅く感じる。

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F

# Create a SparkSession。pythonからsparkを使う場合、セッションの作成が必要。
spark = SparkSession.builder.appName("Testing PySpark Example").getOrCreate()

# デフォルトのログレベルだと大量にログが出力されるので限定する。
spark.sparkContext.setLogLevel("ERROR") 

In [2]:
sample_data = [{"name": "John    D.", "age": 30},
  {"name": "Alice   G.", "age": 25},
  {"name": "Bob  T.", "age": 35},
  {"name": "Eve   A.", "age": 28}]

df = spark.createDataFrame(sample_data)

In [10]:

# Remove additional spaces in name
def remove_extra_spaces(df, column_name):
    # Remove extra spaces from the specified column
    df_transformed = df.withColumn(column_name, F.regexp_replace(col(column_name), "\\s+", " "))

    return df_transformed

transformed_df = remove_extra_spaces(df, "name")

transformed_df.show()

+---+--------+
|age|    name|
+---+--------+
| 30| John D.|
| 25|Alice G.|
| 35|  Bob T.|
| 28|  Eve A.|
+---+--------+



## 色々お試し

### 下記のような読み込み方はダメ

In [ ]:
sample_data_2 = {
    "name" : ["John    D.", "Alice   G.", "Bob  T.", "Eve   A."],
    "age" : [30, 25, 35, 28]
}
# [{"name": "John    D.", "age": 30},
#   {"name": "Alice   G.", "age": 25},
#   {"name": "Bob  T.", "age": 35},
#   {"name": "Eve   A.", "age": 28}]

aaa = spark.createDataFrame(sample_data_2)

PySparkTypeError: [CANNOT_INFER_SCHEMA_FOR_TYPE] Can not infer schema for type: `str`.

### 比較的楽なサンプルデータの作成方法

In [5]:
df_sample = spark.createDataFrame([
    (1, 2., 'string1', 'AAA', 1000),
    (2, 3., 'string2', 'BBB', 2000),
    (3, 4., 'string3', 'AAA', 300),
    (4, 5., 'string4', 'BBB', 100),
    (5, 6., 'string5', 'CCC', 999),
], schema='a long, b double, c string, d string, e long')
df_sample.show()

+---+---+-------+---+----+
|  a|  b|      c|  d|   e|
+---+---+-------+---+----+
|  1|2.0|string1|AAA|1000|
|  2|3.0|string2|BBB|2000|
|  3|4.0|string3|AAA| 300|
|  4|5.0|string4|BBB| 100|
|  5|6.0|string5|CCC| 999|
+---+---+-------+---+----+



### 列を足す

In [12]:
# 列を足す。
transformed_df.withColumn(
    "new_column_1", F.col("age") + 3
).show()

+---+--------+------------+
|age|    name|new_column_1|
+---+--------+------------+
| 30| John D.|          33|
| 25|Alice G.|          28|
| 35|  Bob T.|          38|
| 28|  Eve A.|          31|
+---+--------+------------+



### フィルタリング

In [13]:
transformed_df.filter(
    F.col("age") >= 30
).show()

+---+-------+
|age|   name|
+---+-------+
| 30|John D.|
| 35| Bob T.|
+---+-------+



### 正規表現で置き換え

In [18]:
transformed_df.select(
    "age",
    F.regexp_replace(F.col("name"), "\s", "_").alias("name_mod")  # 半角スペースをアンダースコアに変換する
).show()

+---+--------+
|age|name_mod|
+---+--------+
| 30| John_D.|
| 25|Alice_G.|
| 35|  Bob_T.|
| 28|  Eve_A.|
+---+--------+



### 文字列のスライシング

In [24]:
transformed_df.select(
    "age",
    F.col("name").alias("Full_name"),
    F.substring(F.col("name"), -2, 2).alias("Last_name")
).show()

+---+---------+---------+
|age|Full_name|Last_name|
+---+---------+---------+
| 30|  John D.|       D.|
| 25| Alice G.|       G.|
| 35|   Bob T.|       T.|
| 28|   Eve A.|       A.|
+---+---------+---------+



### 文字列の分割

In [31]:
transformed_df.select(
    "age",
    F.split("name", " ")[0].alias("First_name"),  # 半角スペースでnameを分割してそれぞれのカラムとする
    F.split("name", " ")[1].alias("Last_name")
).show()

+---+----------+---------+
|age|First_name|Last_name|
+---+----------+---------+
| 30|      John|       D.|
| 25|     Alice|       G.|
| 35|       Bob|       T.|
| 28|       Eve|       A.|
+---+----------+---------+



### 集計
これもほぼPolarsと変わらない。

In [33]:
df_sample.show()

+---+---+-------+---+----+
|  a|  b|      c|  d|   e|
+---+---+-------+---+----+
|  1|2.0|string1|AAA|1000|
|  2|3.0|string2|BBB|2000|
|  3|4.0|string3|AAA| 300|
|  4|5.0|string4|BBB| 100|
|  5|6.0|string5|CCC| 999|
+---+---+-------+---+----+



In [8]:
df_sample.groupBy("d").agg(
    F.sum("e")
).show()

+---+------+
|  d|sum(e)|
+---+------+
|AAA|  1300|
|BBB|  2100|
|CCC|   999|
+---+------+



### 比較演算子を用いた2つのDataframeの結合

In [14]:
df_1 = spark.createDataFrame([
    ('ID_1', 1000),
    ('ID_2', 2000),
    ('ID_3', 300),
    ('ID_4', 100),
    ('ID_5', 200),
], schema='a1 string, a2 long').withColumns({
    "a2_max": F.col("a2") + 200,
    "a2_min": F.col("a2") - 200,
})

df_1.show()

+----+----+------+------+
|  a1|  a2|a2_max|a2_min|
+----+----+------+------+
|ID_1|1000|  1200|   800|
|ID_2|2000|  2200|  1800|
|ID_3| 300|   500|   100|
|ID_4| 100|   300|  -100|
|ID_5| 200|   400|     0|
+----+----+------+------+



In [15]:
df_2 = spark.createDataFrame([
    ('apple', 'ID_1', 900),
    ('meat', 'ID_9', 2000),
    ('fish', 'ID_3', 10000),
    ('lemon', 'ID_10', 800),
    ('potato', 'ID_13', 0),
], schema='b1 string, b2 string, b3 long')

df_2.show()

+------+-----+-----+
|    b1|   b2|   b3|
+------+-----+-----+
| apple| ID_1|  900|
|  meat| ID_9| 2000|
|  fish| ID_3|10000|
| lemon|ID_10|  800|
|potato|ID_13|    0|
+------+-----+-----+



In [16]:
df_2.join(
    df_1,
    [
        df_2.b3 < df_1.a2_max,
        df_2.b3 > df_1.a2_min,
    ],
    "inner"
).show()

+------+-----+----+----+----+------+------+
|    b1|   b2|  b3|  a1|  a2|a2_max|a2_min|
+------+-----+----+----+----+------+------+
| apple| ID_1| 900|ID_1|1000|  1200|   800|
|  meat| ID_9|2000|ID_2|2000|  2200|  1800|
|potato|ID_13|   0|ID_4| 100|   300|  -100|
+------+-----+----+----+----+------+------+



上記のようにjoinでカラムを指定する箇所に比較演算子の形で書くことで、  
ある程度数値の幅があっても結合することが出来る。  
上記の場合、df_2のb3の値がa2の最小値より大きく、最大値よりも小さい範囲にある行を結合する形となる。  

これは例えば、ある人が申告した事象発生時刻と実際にシステムでその事象が記録された時刻にぶれがある場合でも両者を紐づけたいときに使える。  